https://towardsdatascience.com/talk-to-your-sql-database-using-langchain-and-azure-openai-bb79ad22c5e2

https://communities.sas.com/t5/SAS-Communities-Library/Exploring-LangChain-and-Azure-OpenAI-s-Ability-to-Write-SQL-and/ta-p/909778

https://python.langchain.com/v0.1/docs/use_cases/sql/agents/#agent

https://python.langchain.com/v0.1/docs/use_cases/sql/



In [ ]:
%pip install --upgrade --quiet  llama-cpp-python langchain-community

Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain_community.utilities import SQLDatabase
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents import create_sql_agent
from langchain_community.llms import LlamaCpp

import warnings
warnings.filterwarnings("ignore")

In [2]:
llm_path = "llm/sqlcoder2.Q3_K_S.gguf"

llm = LlamaCpp(
    model_path=llm_path,
    temperature=0.1,
    n_ctx=4096,
    verbose=True,
)

llama_model_loader: loaded meta data with 19 key-value pairs and 485 tensors from llm/sqlcoder2.Q3_K_S.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = starcoder
llama_model_loader: - kv   1:                               general.name str              = StarCoder
llama_model_loader: - kv   2:                   starcoder.context_length u32              = 8192
llama_model_loader: - kv   3:                 starcoder.embedding_length u32              = 6144
llama_model_loader: - kv   4:              starcoder.feed_forward_length u32              = 24576
llama_model_loader: - kv   5:                      starcoder.block_count u32              = 40
llama_model_loader: - kv   6:             starcoder.attention.head_count u32              = 48
llama_model_loader: - kv   7:          starcoder.attention.head_count_kv u32              

In [3]:
db = SQLDatabase.from_uri("sqlite:///Chinook.db", sample_rows_in_table_info=0)
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artist LIMIT 10;")

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

In [4]:
context = db.get_context()
print(list(context))
print(context["table_info"])

['table_info', 'table_names']

CREATE TABLE "Album" (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")
)


CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)


CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60) NOT NULL, 
	"SupportRepId" INTEGER, 
	PRIMARY KEY ("CustomerId"), 
	FOREIGN KEY("SupportRepId") REFERENCES "Employee" ("EmployeeId")
)


CREATE TABLE "Employee" (
	"EmployeeId" INTEGER NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"FirstName" NVARCHAR(20) NOT NULL, 
	"Title" NVARCHAR(30), 
	"Repo

In [5]:
context

{'table_info': '\nCREATE TABLE "Album" (\n\t"AlbumId" INTEGER NOT NULL, \n\t"Title" NVARCHAR(160) NOT NULL, \n\t"ArtistId" INTEGER NOT NULL, \n\tPRIMARY KEY ("AlbumId"), \n\tFOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")\n)\n\n\nCREATE TABLE "Artist" (\n\t"ArtistId" INTEGER NOT NULL, \n\t"Name" NVARCHAR(120), \n\tPRIMARY KEY ("ArtistId")\n)\n\n\nCREATE TABLE "Customer" (\n\t"CustomerId" INTEGER NOT NULL, \n\t"FirstName" NVARCHAR(40) NOT NULL, \n\t"LastName" NVARCHAR(20) NOT NULL, \n\t"Company" NVARCHAR(80), \n\t"Address" NVARCHAR(70), \n\t"City" NVARCHAR(40), \n\t"State" NVARCHAR(40), \n\t"Country" NVARCHAR(40), \n\t"PostalCode" NVARCHAR(10), \n\t"Phone" NVARCHAR(24), \n\t"Fax" NVARCHAR(24), \n\t"Email" NVARCHAR(60) NOT NULL, \n\t"SupportRepId" INTEGER, \n\tPRIMARY KEY ("CustomerId"), \n\tFOREIGN KEY("SupportRepId") REFERENCES "Employee" ("EmployeeId")\n)\n\n\nCREATE TABLE "Employee" (\n\t"EmployeeId" INTEGER NOT NULL, \n\t"LastName" NVARCHAR(20) NOT NULL, \n\t"FirstName" NVA

In [6]:
# Set up SQL toolkit for LangChain Agent  
toolkit = SQLDatabaseToolkit(db=db, llm=llm)  

In [7]:
# Initialize and run the Agent  
agent_executor = create_sql_agent(  
    llm=llm,  
    toolkit=toolkit,  
    verbose=True,
    handle_parsing_errors=True,
    handle_sql_errors=True
)

In [10]:
def format_prompt(question):
    return f"""
        create a sql query to answers this question:
        {question}        

        finally return an answer based on the db data
    """

In [ ]:
agent_executor.invoke(format_prompt("show an artist name starting with Q"))



> Entering new SQL Agent Executor chain...


Llama.generate: 601 prefix-match hit, remaining 46 prompt tokens to eval


In [ ]:
agent_executor.run("how many tracks were composed by 'AC/DC'?")



> Entering new SQL Agent Executor chain...
Action: sql_db_schema
Action Input: "track", album, artistError: table_names {'artist', 'track"', 'album'} not found in database The error message indicates that there are some missing tables or fields. Let me try to query for just the track field from each of those tables.
Action: sql_db_schema
Action Input: "track", artist, albumError: table_names {'artist', 'track"', 'album'} not found in database I should use sql_db_list_tables instead and then check which fields are available for the track field from each of those tables.
Action: sql_db_list_tables
Action Input: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track Now that I know what columns exist in the database, let me try to query for just the track field from each of those tables.
Action: sql_db_schema
Action Input: "track", artist, albumError: table_names {'artist', 'track"', 'album'} not found in database I should use sql_db_li

In [10]:
agent_executor.run("how many artists are?")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input:Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track The table that contains artist information is 'Artist'. Now I need to find out what columns are in the 'Artist' table.
Action: sql_db_schema
Action Input: Artist
CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
1 rows from Artist table:
ArtistId	Name
1	AC/DC
*/ The 'Artist' table has only two columns. I can query it directly.
Action: sql_db_query
Action Input: SELECT Name FROM Artist LIMIT 10[('AC/DC',), ('Accept',), ('Aerosmith',), ('Alanis Morissette',), ('Alice In Chains',), ('Antônio Carlos Jobim',), ('Apocalyptica',), ('Audioslave',), ('BackBeat',), ('Billy Cobham',)] I now know the number of artists in my database.
Final Answer: There are 10 artists.

> Finished chain.


'There are 10 artists.'

In [25]:
agent_executor.run("what's the name of the person that made the most expensive purchase")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: ""Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, TrackThe relevant tables for this query seem to be the Customer and Invoice tables. I should check their schemas to see what information they contain.
Action: sql_db_schema
Action Input: Customer, Invoice
CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60) NOT NULL, 
	"SupportRepId" INTEGER, 
	PRIMARY KEY ("CustomerId"), 
	FOREIGN KEY("SupportRepId") REFERENCES "Employee" ("EmployeeId")
)

/*
3 rows from Customer table:
CustomerId	FirstName	LastName	Company	Address	City	State	Country	PostalCode	Phone	Fax	

'The person who made the most expensive purchase is Helena Holý.'